https://metanit.com/sharp/tutorial/11.1.php

In [1]:
using System.Threading;
 
// получаем текущий поток
thread_info(Thread.CurrentThread);

void thread_info(Thread currentThread){
    //получаем имя потока
    Console.WriteLine($"Имя потока: {currentThread.Name}");
    currentThread.Name = "Метод Main";
    Console.WriteLine($"Имя потока: {currentThread.Name}");
    
    Console.WriteLine($"Запущен ли поток: {currentThread.IsAlive}");
    Console.WriteLine($"Id потока: {currentThread.ManagedThreadId}");
    Console.WriteLine($"Приоритет потока: {currentThread.Priority}");
    Console.WriteLine($"Статус потока: {currentThread.ThreadState}");
    Console.WriteLine("--------------------------------------");
    Console.WriteLine();
}

Имя потока: .NET ThreadPool Worker
Имя потока: Метод Main
Запущен ли поток: True
Id потока: 6
Приоритет потока: Normal
Статус потока: Background
--------------------------------------



In [2]:
using System.Threading;
 
for(int i = 0; i < 10; i++)
{
    Thread.Sleep(500);      // задержка выполнения на 500 миллисекунд
    Console.WriteLine(i);
}

0
1
2
3
4
5
6
7
8
9


In [11]:
using System.Threading;

// public delegate void ThreadStart();

// создаем новый поток
Thread myThread1 = new Thread(Print); 
Thread myThread2 = new Thread(new ThreadStart(Print));
Thread myThread3 = new Thread(()=>Console.WriteLine("Hello Threads"));

thread_info(myThread1);
thread_info(myThread2);
thread_info(myThread3);
 
myThread1.Start();  // запускаем поток myThread1
myThread2.Start();  // запускаем поток myThread2
myThread3.Start();  // запускаем поток myThread3

void Print() => Console.WriteLine("Hello Threads");

Имя потока: 
Имя потока: Метод Main
Запущен ли поток: False
Id потока: 38
Приоритет потока: Normal
Статус потока: Unstarted
--------------------------------------

Имя потока: 
Имя потока: Метод Main
Запущен ли поток: False
Id потока: 39
Приоритет потока: Normal
Статус потока: Unstarted
--------------------------------------

Имя потока: 
Имя потока: Метод Main
Запущен ли поток: False
Id потока: 40
Приоритет потока: Normal
Статус потока: Unstarted
--------------------------------------

Hello Threads
Hello Threads
Hello Threads


Преимуществом потоков является то, что они могут выполняться одновременно. Например:

In [12]:
using System.Threading;
 
// создаем новый поток
Thread myThread = new Thread(Print);
// запускаем поток myThread
myThread.Start();
 
// действия, выполняемые в главном потоке
for (int i = 0; i < 5; i++)
{
    Console.WriteLine($"Главный поток: {i}");
    Thread.Sleep(300);
}
 
// действия, выполняемые во втором потокке
void Print()
{
    for (int i = 0; i < 5; i++)
    {
        Console.WriteLine($"Второй поток: {i}");
        Thread.Sleep(400);
    }
}

Главный поток: 0
Второй поток: 0
Главный поток: 1
Второй поток: 1
Главный поток: 2
Второй поток: 2
Главный поток: 3
Второй поток: 3
Главный поток: 4


## Потоки с параметрами

In [13]:
using System.Threading;

void Print(object? message) => Console.WriteLine(message);

// создаем новые потоки
Thread myThread1 = new Thread(new ParameterizedThreadStart(Print));
Thread myThread2 = new Thread(Print);
Thread myThread3 = new Thread(message => Console.WriteLine(message));
 
// запускаем потоки
myThread1.Start("Hello");
myThread2.Start("Привет");
myThread3.Start("Salut");

Error: (7,24): error CS0121: The call is ambiguous between the following methods or properties: 'Thread.Thread(ThreadStart)' and 'Thread.Thread(ParameterizedThreadStart)'

### Передаём разные типы в поток

Передаём тип int

In [14]:
using System.Threading;
 
int number = 4;
// создаем новый поток
Thread myThread = new Thread(Print);
myThread.Start(number);    // n * n = 16
 
 
// действия, выполняемые во втором потокке
void Print(object? obj)
{
    // здесь мы ожидаем получить число
    if (obj is int n)
    {
        Console.WriteLine($"n * n = {n * n}");
    }
}

Error: (5,23): error CS0121: The call is ambiguous between the following methods or properties: 'Thread.Thread(ThreadStart)' and 'Thread.Thread(ParameterizedThreadStart)'

Создаём собственный тип (передаём параметры разного типа)

In [15]:
using System.Threading;
 
Person tom = new Person("Tom", 37);
// создаем новый поток
Thread myThread = new Thread(Print);
myThread.Start(tom);
 
void Print(object? obj)
{
    // здесь мы ожидаем получить объект Person
    if (obj is Person person)
    {
        Console.WriteLine($"Name = {person.Name}");
        Console.WriteLine($"Age = {person.Age}");
    }
}
 
record class Person(string Name, int Age);

Error: (5,23): error CS0121: The call is ambiguous between the following methods or properties: 'Thread.Thread(ThreadStart)' and 'Thread.Thread(ParameterizedThreadStart)'

Но тут опять же есть одно ограничение: метод Thread.Start не является типобезопасным, то есть мы можем передать в него любой тип, и потом нам придется приводить переданный объект к нужному нам типу. Для решения данной проблемы рекомендуется объявлять все используемые методы и переменные в специальном классе, а в основной программе запускать поток через ThreadStart. Например:

In [16]:
using System.Threading;
 
Person tom = new Person("Tom", 37);
// создаем новый поток
Thread myThread = new Thread(tom.Print);
myThread.Start();
 
record class Person(string Name, int Age)
{
    public void Print()
    {
        Console.WriteLine($"Name = {Name}");
        Console.WriteLine($"Age = {Age}");
    }
}

Name = Tom
Age = 37


## Синхронизация потоков

In [17]:
using System.Threading;

int x = 0;
 
// запускаем пять потоков
for (int i = 1; i < 6; i++)
{
    Thread myThread = new (Print);
    myThread.Name = $"Поток {i}";   // устанавливаем имя для каждого потока
    myThread.Start();
}
 
void Print()
{
    x = 1;
    for (int i = 1; i < 6; i++)
    {
        Console.WriteLine($"{Thread.CurrentThread.Name}: {x}");
        x++;
        Thread.Sleep(100);
    }
}

Поток 1: 1
Поток 4: 1
Поток 3: 1
Поток 2: 1
Поток 5: 1


In [18]:
int x = 0;
object locker = new();  // объект-заглушка
// запускаем пять потоков
for (int i = 1; i < 6; i++)
{
    Thread myThread = new(new ThreadStart(Print));
    myThread.Name = $"Поток {i}";
    myThread.Start();
}
 
 
void Print()
{
    lock (locker)
    {
        x = 1;
        for (int i = 1; i < 6; i++)
        {
            Console.WriteLine($"{Thread.CurrentThread.Name}: {x}");
            x++;
            Thread.Sleep(100);
        }
    }
}

Поток 1: 1
